In [1]:
import pandas as pd
from datetime import datetime

# entity_df generally comes from upstream systems
entity_df = pd.DataFrame.from_dict({
    "driver_id": [1001, 1002, 1003, 1004],
    "event_timestamp": [
        datetime(2021, 4, 12, 10, 59, 42),
        datetime(2021, 4, 12, 8,  12, 10),
        datetime(2021, 4, 12, 16, 40, 26),
        datetime(2021, 4, 12, 15, 1 , 12)
    ]
})

entity_df.head()

,driver_id,event_timestamp
0,1001,2021-04-12 10:59:42
1,1002,2021-04-12 08:12:10
2,1003,2021-04-12 16:40:26
3,1004,2021-04-12 15:01:12


In [3]:
from feast import FeatureStore

store = FeatureStore(repo_path="../feature_repo")

training_df = store.get_historical_features(
    entity_df=entity_df, 
    feature_refs = [
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
).to_df()

training_df.head()

/Users/ishidashuhei/.pyenv/versions/3.7.9/envs/feast/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


,event_timestamp,driver_id,driver_hourly_stats__conv_rate,driver_hourly_stats__acc_rate,driver_hourly_stats__avg_daily_trips
0,2021-04-12 08:12:10+00:00,1002,0.395467,0.869110,746
1,2021-04-12 10:59:42+00:00,1001,0.767152,0.825250,730
2,2021-04-12 15:01:12+00:00,1004,0.569010,0.327007,218
3,2021-04-12 16:40:26+00:00,1003,0.102078,0.654755,866


In [5]:
from pprint import pprint
from feast import FeatureStore

store = FeatureStore(repo_path="../feature_repo")

feature_vector = store.get_online_features(
    feature_refs=[
        'driver_hourly_stats:conv_rate',
        'driver_hourly_stats:acc_rate',
        'driver_hourly_stats:avg_daily_trips'
    ],
    entity_rows=[{"driver_id": 1001}]
).to_dict()

pprint(feature_vector)

{'driver_hourly_stats__acc_rate': [0.6285330653190613],
 'driver_hourly_stats__avg_daily_trips': [713],
 'driver_hourly_stats__conv_rate': [0.7606037855148315],
 'driver_id': [1001]}


In [6]:
features = pd.read_parquet('../feature_repo/data/driver_stats.parquet')

/Users/ishidashuhei/.pyenv/versions/3.7.9/envs/feast/lib/python3.7/site-packages/ipykernel/ipkernel.py:283: DeprecationWarning: `should_run_async` will not call `transform_cell` automatically in the future. Please pass the result to `transformed_cell` argument and any exception that happen during thetransform in `preprocessing_exc_tuple` in IPython 7.17 and above.
  and should_run_async(code)


In [8]:
features.query('driver_id == 1001')

,datetime,driver_id,conv_rate,acc_rate,avg_daily_trips,created
1444,2021-04-24 21:00:00+00:00,1001,0.830319,0.592802,54,2021-05-09 21:50:04.097
1445,2021-04-24 22:00:00+00:00,1001,0.298055,0.123848,420,2021-05-09 21:50:04.097
1446,2021-04-24 23:00:00+00:00,1001,0.372776,0.661141,389,2021-05-09 21:50:04.097
1447,2021-04-25 00:00:00+00:00,1001,0.732662,0.485940,722,2021-05-09 21:50:04.097
1448,2021-04-25 01:00:00+00:00,1001,0.526828,0.611015,155,2021-05-09 21:50:04.097
...,...,...,...,...,...,...
1800,2021-05-09 17:00:00+00:00,1001,0.235436,0.529497,816,2021-05-09 21:50:04.097
1801,2021-05-09 18:00:00+00:00,1001,0.282028,0.329207,907,2021-05-09 21:50:04.097
1802,2021-05-09 19:00:00+00:00,1001,0.575781,0.521369,873,2021-05-09 21:50:04.097
1803,2021-05-09 20:00:00+00:00,1001,0.555681,0.726412,173,2021-05-09 21:50:04.097
